## EuropePMC API for remainder

This will get as many of the reamaining unmapped PMIDs as possible from the EurpoPMC API.

In [1]:
import pandas as pd
from tqdm import tqdm
import pickle
import requests
import json

In [2]:
still_no_map = pickle.load(open('../data/no_map_NLM.pkl', 'rb'))
new_map = {}

In [3]:
URL="https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=ext_id:{}%20src:med&format=json"

In [4]:
def get_date(result):
    res = result['resultList']['result']
    if len(res) == 1 and type(res[0]) == dict:
        return res[0].get('firstPublicationDate', None)
    elif len(res) > 1 and type(res[0]) == dict:
        dates = []
        for r in res:
            date = res[0].get('firstPublicationDate', '9999-99-99')
            dates.append(date)
        return min(dates)
    
    return None

Saving this for last because I don't want to hammer the API... but... looking at the docs:

https://europepmc.org/RestfulWebService

I don't see a way to do a batch query...

In [5]:
# Slow, but only way to get is to query API individually...
for pmid in tqdm(still_no_map):
    r = requests.get(URL.format(pmid))
    result = json.loads(r.text)
    date = get_date(result)
    new_map[pmid] = date

100%|██████████| 14048/14048 [2:33:38<00:00,  1.53it/s] 


In [6]:
new_map = {k: v for k, v in new_map.items() if v is not None}
print('{:,}'.format(len(new_map)))

final_no_map = still_no_map - set(new_map.keys())
print('{:,}'.format(len(final_no_map)))

7,937
6,111


Got mappings for another 7,937 PMIDs... great!

In [7]:
pickle.dump(new_map, open('../data/pmid_to_year_EBI.pkl', 'wb'))
pickle.dump(final_no_map, open('../data/no_map_EBI.pkl', 'wb'))